# Classificació de cel·lules emprant la U-Net

La xarxa U-Net és emprada per la generació de segmentacions, en particular en l'ambit mèdic. Introduïda per primer cop per *Ronneberger et al.*, aquesta xarxa es caracteritza per una estructura de codificador-decodificador i un conjunt de connexions saltejades. Aquestes característiques provoquen que sigui un model simple alhora que potent.

<img style="width:50%" src="https://lmb.informatik.uni-freiburg.de/Publications/2015/RFB15a/u-net-architecture.png" />

L'entrenament d'aquesta xarxa és el que s'anomena de cap a cap, això és així ja que tota la xarxa s'entrena de cop. Aquest fet és un dels que simplifiquen l'entrenament i permet l'ús de tècniques de augmentació de dades sense problemes.

In [ ]:
import glob
import os
import json
import re

from numpy.random import seed
from matplotlib import pyplot as plt
import numpy as np
import cv2
import skimage
import skimage.color
import skimage.io
import skimage.transform
import imgaug as ia
import imgaug.augmenters as iaa

# Llibraries pròpies
from u_cells.u_cells.unet import data as u_data
from u_cells.u_cells.unet import model as u_model

seed(1)

## Descarregam les dades

**IMPORTANT**: Només executar aquesta cel·la la primera vegada que s'executa la xarxa

In [ ]:
if False:
    # !wget -O dataset_four.zip https://www.dropbox.com/s/0v6rdf3xhoge0vh/unet_color_quatre.zip?dl=1 
    # !unzip dataset_four.zip > /dev/null
    !wget -O nuclei_data.zip https://www.dropbox.com/s/wllznkxlw5jdj9u/nuclei_data.zip?dl=1
    !unzip nuclei_data.zip > /dev/null

## U-Net sense modificar

### Paràmetres 

En aquesta cel·la definim tot un conjunt de paràmetres que emprarem per la xarxa. 

In [ ]:
BATCH_SIZE = 4
TOTAL_IMAGES = 300
STEPS_PER_EPOCH = TOTAL_IMAGES // BATCH_SIZE 
EPOCHS = 20

TRAIN_PATH = './in/bboxes_class/train'
TEST_PATH = './in/bboxes_class/val'

RESET_DATA = False

En aquesta secció construeix entrena i testeja la xarxa U-Net original tal com es proposada per Ronnenberger et al.

### Preparació de les dades i generació de les dades

#### Generació de les dades

La llibreria d'augmentació de dades s'executa a la CPU. Això suposa un coll de botella per l'entrenament, per tal de resoldre-ho el que hem fet és generar primer totes les imatges que emprarem, guardar-les a disc i després llegir-les. Això redueix considerablement el temps d'execució i a més permet un millor control del que passa, ja que tenim accés a les imatges.

Per fer-ho empram la llibreria **ImgAug** de *Jung et al.*

In [ ]:
if RESET_DATA:
    sometimes = lambda aug: iaa.Sometimes(0.5, aug)

    augmentation = [  # apply the following augmenters to most images
        iaa.Fliplr(0.5),  # horizontally flip 50% of all images
        iaa.Flipud(0.2),  # vertically flip 20% of all images

        sometimes(iaa.Affine(
                scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
                # scale images to 80-120% of their size, individually per axis
                translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)},
                # translate by -20 to +20 percent (per axis)
                rotate=(-45, 45),  # rotate by -45 to +45 degrees
                shear=(-16, 16),  # shear by -16 to +16 degrees
                order=[0, 1],  # use nearest neighbour or bilinear interpolation (fast)
                cval=0,  # if mode is constant, use a cval between 0 and 255
                mode=ia.ALL
                # use any of scikit-image's warping modes (see 2nd image from the top for examples)
            )),

         iaa.SomeOf((0, 5),
                   [
                       iaa.OneOf([
                           iaa.GaussianBlur((0, 3.0)),  # blur images with a sigma between 0 and 3.0
                           iaa.AverageBlur(k=(2, 7)),
                           # blur image using local means with kernel sizes between 2 and 7
                           iaa.MedianBlur(k=(3, 11)),
                           # blur image using local medians with kernel sizes between 2 and 7
                           ]),
                   ],
                   random_order=True)]

    u_data.generate_data(BATCH_SIZE * STEPS_PER_EPOCH, './in/bboxes_class/train/*.png', './out_aug/train/', augmentation, './in/bboxes_class/train/via_region_data.json', to_mask = True, output_shape=(512, 512))

### Generadors de Keras

Una vegada hem creat i guardat les imatges augmentades a disc empram els generadors de keras per llegir la informació d'entrenament i validació. 

In [ ]:
train_generator = u_data.DataGenerator(BATCH_SIZE, STEPS_PER_EPOCH, './out_aug/train/*.png', (512, 512), 
                                       output_size = 100, augmentation=None, load_from_cache = True, 
                                       do_background = True, multi_type = True, binary_output = True)
val_generator = u_data.DataGenerator(4, 10, TEST_PATH, (512, 512), output_size =  100, 
                                     region_path="via_region_data.json", do_background = True, 
                                     multi_type = True, augmentation=None, binary_output = True)

In [ ]:
for t, m in train_generator:
    break

In [ ]:
plt.figure()
plt.imshow(m['img_out'][0,:,:,0])

## Cream el model

Cream el model de la UNet, per fer-ho definim la mida de l'entrada, de la sortida i un parell de flags que determinen que farà la xarxa. 

A més també definim indicam la funció de pèrdua que emprarem i el nombre de filtres. Finalment també el nombre de blocs ( el mateix nombre pel codificador que pel descodificador ) 

In [ ]:
model = u_model.UNet(input_size=(None,None,3), out_channel=2, batch_normalization=False)

model.build_unet(n_filters=64, dilation_rate=1, layer_depth = 5, last_activation="sigmoid")
model.compile(loss_func = clever_categorical, learning_rate = 3e-6, run_eagerly=True)

Ens interessa saber la forma del model. En particular la mida d'entrada i sortida de cada una de les capes, per tant feim una visualització per defecte de la llibreria.

### Entrenament
---

Entrament de la xarxa. Els valors per defecte són 100 steps per validació, 300 per època d'entrenament i 10 èpoques.

In [ ]:
model.train(train_generator, val_generator, epochs=EPOCHS, steps_per_epoch=STEPS_PER_EPOCH, 
            check_point_path=None, validation_steps=2 )

In [ ]:
history = model.history

plt.figure(figsize=(9,6), dpi= 100, facecolor='w', edgecolor='k')

plt.subplot(1,2,1)
plt.plot(history.history['categorical_accuracy'])
plt.plot(history.history['val_categorical_accuracy'])
plt.title('Model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')

# "Loss"
plt.subplot(1,2,2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

### Carregam un model preentrenat

Podem carregar un model ja entrenat, per fer-ho el que necessitam es primer generar el model i després carragar-hi els pesos.

*TODO: Automàticament selecionar els darrers pesos de la carpeta.*

In [ ]:
model.load_weights('./out/model.02-0.02.h5')

### Resultats - avaluació
---
Una vegada entrenat un model podem visualitzar els resulats, per fer-ho primer de tot cream un generador i després el feim prediccions. Obtenim també l'**IOU**. Definim un generador d'imatges de *test*. La seva principal característica es que no retorna cap GT.

In [ ]:
info = json.load(open('./in/bboxes_class/val/via_region_data.json'))
info = {k: (len(v["regions"])) for k, v in info.items()}

def testGenerator(test_path, target_size=(256, 256), flag_multi_class=False,
                  as_gray=True, image=True):

    filenames = glob.glob(test_path)
    filenames.sort(key=lambda f: int(re.sub('\D', '', f)))
    # filenames = sorted(filenames)
    for filename in filenames:
        path, name = os.path.split(filename)
        print(f"{filename=}")
        img = cv2.imread(filename)
        img = img
        img = skimage.transform.resize(img, target_size)
        img = img.reshape(1, target_size[0], target_size[1], 3)

        yield img

In [ ]:
testGene = testGenerator('./in/bboxes_class/val/*.png', target_size=(256, 256, 3), as_gray = False)
masks  = model.model.predict(testGene, 16)

In [ ]:
%matplotlib notebook

plt.figure(figsize=(8,5), dpi= 100, facecolor='w', edgecolor='k')
mask_a = masks.astype(np.float64)
suma = np.sum(mask_a[0, :, :, 0:1], axis=2)
plt.imshow(suma)
plt.show()

### *Fine tunning*

Realitzam fine tunning. Per fer-ho congelam les primeres dotze capes i a partir de la 17 fins a la tercera començan pel final, l'objectiu: mantenir les features apreses i alhora refinar els resultats pel nostre dataset.

In [ ]:
for l in model.model.layers:
    l.trainable = True

In [ ]:
for l in model.model.layers[:10]:
    l.trainable = False
    
for l in model.model.layers[20:-3]:
    l.trainable = False

In [ ]:
train_generator = u_data.DataGenerator(BATCH_SIZE, STEPS_PER_EPOCH, './out_aug/train/*.png', (256, 256), 100, augmentation=None, load_from_cache = True, do_background = True, multi_type = True)

In [ ]:
K.set_value(model.model.optimizer.learning_rate, 3e-8) # Indicam un lr molt petit
model.model.fit(train_generator, epochs=5, steps_per_epoch=75)

### Guardam els resultats

Guardam els resultats amb un fitxer per canal

In [ ]:
for i in range(0,5):
    for n_channel in range(0, masks.shape[3]):
        path = os.path.join(".", "out", "ccce", "img_" + str(i).zfill(2))
        os.makedirs(path, exist_ok = True)
        
        mask = masks[i, :, :, n_channel]
        mask = mask / mask.max()
        
        cv2.imwrite(os.path.join(path, str(n_channel).zfill(4) + ".png"), mask * 255)

## Brutor

In [ ]:
plt.rcParams["figure.figsize"] = (7,7)

In [ ]:
def remove_border_cells(contours, shape):
    """
    Removes the objects from the borders of the image.

    A border of an image is the zone near the start or the end of the matrix. 
    The index of this points are near 0 and near the shape of the image. The 
    contours checked as a parameter don't has that exactly index so is needed 
    to has an acceptable error.

    Args:
        contours: List of numpy arrays, every array its a different contour. The array has two
                  columns and many rows as points in the contour. Depending of the appoximation
                  method used
        shape:

    Returns:

    """
    center_contours = []

    for cont in contours:
        cont = np.squeeze(cont)
        border = not np.all(
            (cont[:, 0] > 15) & (cont[:, 1] > 15) & (cont[:, 0] < shape[1] - 15) &
            (cont[:, 1] < shape[0] - 15))

        if not border:
            center_contours.append(np.array([cont]).reshape((cont.shape[0], 1, 
                                                             cont.shape[1])))

    return center_contours

def get_iou(ground, prediction, th, debug=False, remove_border_segs = False):
    assert ground.shape[2] == prediction.shape[2]
  
    ground = ground.astype(np.float32) / ground.max()
    ious = [0] * ground.shape[2]
  
    if debug:
        fig = plt.figure(1,(16, 12))
        idx = 1

    for channel_idx in range(0, ground.shape[2]):

        channel_gt = ground[:,:, channel_idx]
        channel_pred = np.copy(prediction[:, :, channel_idx])

        channel_pred = cv2.resize(channel_pred, (channel_gt.shape[1], channel_gt.shape[0]), interpolation = cv2.INTER_NEAREST) 

        channel_pred[channel_pred <= th] = 0
        channel_pred[channel_pred > th] = 1

    if remove_border_segs:
        contours, _ = cv2.findContours((channel_pred * 255).astype(np.uint8), cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
        contours = remove_border_cells(contours, channel_pred.shape)

        channel_pred = np.zeros_like(channel_pred)
        channel_pred = cv2.drawContours(channel_pred, contours, -1, 1, -1)
 
    intersection = cv2.bitwise_and(channel_gt, channel_pred)
    intersection_area = np.count_nonzero(intersection)
    
    union = cv2.bitwise_or(channel_gt, channel_pred)
    union_area = np.count_nonzero(union)
    
    if debug:
        l = [channel_gt, channel_pred, intersection, union, (union-intersection)]
        titles = ["GT", "PRED", "INTERSEC", "UNION", "DIFF"]
    
        for i in range(len(l)):
            plt.title(titles[(i-1) % 5])
            plt.subplot(4,5, idx)
            plt.imshow(l[i], cmap="gray");
            idx += 1
    iou = 0
    if union_area > 0:
        iou = round(intersection_area / union_area, 3)

    ious[channel_idx] += iou
    
    return ious

In [ ]:
alpha = 0.5

ious = []
for idx, res in enumerate(results):
    gt_image = cv2.imread("./unet_color_quatre/test/label/" + str(idx) + ".png")
    # cv2 reads the images in BGR we convert them into rgb
    gt_image = cv2.cvtColor(gt_image, cv2.COLOR_BGR2RGB) 

    gt_image = decode(gt_image, decode_mode.CELLS_BCK)

    iou = get_iou(gt_image, res, alpha)
    ious.append(iou)

    if idx < 10:
        print("Image 0"+ str(idx) + ": " + str(iou))
    else:
        print("Image "+ str(idx) + ": " + str(iou))

print("###########################################")
print("Mean: ", np.mean(ious, axis = 0))

In [ ]:
%matplotlib notebook

plt.imshow(res[:,:,0]);

La següent cel·la només serveix per evaluar el funcionament de _get_iou_

In [ ]:
index = 1

ground = cv2.imread("./unet_color_quatre/test/label/" + str(index) + ".png")
ground = cv2.cvtColor(ground, cv2.COLOR_BGR2RGB) 

prediction = results[index]
th = 0.5
ground = decode(ground)
get_iou(ground, prediction, th, True)